In [2]:
# CPU-only mode (CURRENTLY DISABLED - using GPU with memory growth instead)
# Uncomment lines below if you want to force CPU instead:

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   # Reduce TF logging
# print("✓ GPU disabled - TensorFlow will use CPU only")

In [3]:
# GPU Configuration with Memory Growth
# This allows TensorFlow to allocate GPU memory as needed instead of grabbing it all at once

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"✓ GPU memory growth enabled for {len(gpus)} GPU(s)")
        print(f"  GPU: {gpus[0].name}")
        
        # Optional: Set a memory limit (uncomment if needed)
        # tf.config.set_logical_device_configuration(
        #     gpus[0],
        #     [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]  # 2GB limit
        # )
        
    except RuntimeError as e:
        print(f"✗ Error configuring GPU: {e}")
        print("  TensorFlow may have already been initialized")
else:
    print("⚠️  No GPU found - will use CPU")

✓ GPU memory growth enabled for 1 GPU(s)
  GPU: /physical_device:GPU:0


---
## ⚠️ IMPORTANT: Restart Kernel Now!

**Before running Cell 2 above, you MUST restart the kernel:**

1. Click `Kernel` → `Restart Kernel`  
   OR press `0, 0` (zero twice quickly)
2. Click "Restart" in the dialog
3. Then run Cell 2 first
4. Then run remaining cells in order

**Why?** TensorFlow is already loaded and can't change GPU settings mid-session.

---

This is My First RNN Model for the Purpose of Learning the working of RNN

We will be Building the most simple Sentimental Analysis Model
Text -> Vector (Embedding) -> RNN Forward Prop

## 🔧 Setup Instructions - GPU Mode with Memory Growth

### Configuration: Using GPU with dynamic memory allocation

**How to run:**

1. **Restart Kernel**: `Kernel` → `Restart Kernel` (or press `0, 0` twice)
2. **Run Cell 2 FIRST** - This enables GPU memory growth
3. **Then run remaining cells in order**

### What's different:
- ✅ GPU will allocate memory **gradually** as needed
- ✅ Batch size reduced to 16 (uses less memory)
- ✅ Using 5000 training samples (manageable size)

### If GPU still fails:
1. Close Chrome/Firefox and other GPU-using apps
2. Try even smaller batch size: change `batch_size=16` to `batch_size=8`
3. Reduce dataset: change `x_train[:5000]` to `x_train[:2000]`
4. Or switch to CPU mode: comment Cell 2, uncomment Cell 1

In [4]:
import numpy as np

docs = [
    'go india',
    'india india',
    'hip hip hurray',
    'jeetega bhai jeetega india jeetega',
    'bharat mata ki jai',
    'kohli kohli',
    'rohit rohit',
    'inquilab zindabad'
]

In [5]:
#Tokenisation
from keras_preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token = '<nothing>')
tokenizer.fit_on_texts(docs)

In [6]:
len(tokenizer.word_index)

15

In [7]:
seq = tokenizer.texts_to_sequences(docs)

In [8]:
seq

[[7, 2],
 [2, 2],
 [4, 4, 8],
 [3, 9, 3, 2, 3],
 [10, 11, 12, 13],
 [5, 5],
 [6, 6],
 [14, 15]]

In [9]:
from keras.utils import pad_sequences
seq = pad_sequences(seq, padding='post')

In [10]:
seq

array([[ 7,  2,  0,  0,  0],
       [ 2,  2,  0,  0,  0],
       [ 4,  4,  8,  0,  0],
       [ 3,  9,  3,  2,  3],
       [10, 11, 12, 13,  0],
       [ 5,  5,  0,  0,  0],
       [ 6,  6,  0,  0,  0],
       [14, 15,  0,  0,  0]], dtype=int32)

In [11]:
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, Flatten

In [12]:
# Model for IMDB sentiment analysis
vocab_size = 10000  # IMDB dataset uses top 10k words by default
embedding_dim = 32
max_length = 200

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(SimpleRNN(32, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.summary()

/media/the-aditya-10/New Volume/Aditya/CODING/AI, ML, DL, DS/RNN-Learning/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1762505499.905933   13617 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 130 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
(x_train, y_train), (X_test, Y_test) = imdb.load_data()

In [14]:
# Pad the IMDB sequences to a fixed length
import numpy as np

def pad_sequences_custom(sequences, maxlen=None, padding='post', truncating='post', value=0):
    """Simple padding function compatible with NumPy 2.0"""
    lengths = [len(s) for s in sequences]
    if maxlen is None:
        maxlen = max(lengths) if lengths else 0
    
    result = np.full((len(sequences), maxlen), value, dtype='int32')
    
    for i, s in enumerate(sequences):
        if len(s) > maxlen:
            if truncating == 'pre':
                trunc = s[-maxlen:]
            else:
                trunc = s[:maxlen]
        else:
            trunc = s
        
        if padding == 'post':
            result[i, :len(trunc)] = trunc
        else:
            result[i, -len(trunc):] = trunc
    
    return result

max_length = 200  # Use first 200 words of each review
x_train = pad_sequences_custom(x_train, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences_custom(X_test, maxlen=max_length, padding='post', truncating='post')

print(f"x_train shape: {x_train.shape}")
print(f"X_test shape: {X_test.shape}")

x_train shape: (25000, 200)
X_test shape: (25000, 200)


In [15]:
model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train with smaller batch size for GPU memory efficiency
# Using subset of data for faster training (increase if you want)
model.fit(x_train[:5000], y_train[:5000], 
          epochs=2, 
          batch_size=16,  # Reduced from 32 to use less GPU memory
          validation_data=(X_test[:1000], Y_test[:1000]))

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7182 - loss: 0.4995 - val_accuracy: 0.5050 - val_loss: 0.7656
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7182 - loss: 0.4995 - val_accuracy: 0.5050 - val_loss: 0.7656
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7054 - loss: 0.5233 - val_accuracy: 0.5260 - val_loss: 0.6949
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7054 - loss: 0.5233 - val_accuracy: 0.5260 - val_loss: 0.6949
